# 05- Google Analytics Work

In [1]:
%ls ../data/ga/

'new_users (foreign) top 5000.csv'*  'uniq_pg_views (foreign) top 5000.csv'*
'pg_views (foreign) top 5000.csv'*


In [11]:
# directories and variables
data_dir = '../data/'
google_dir = data_dir + 'ga/'

new_users_f = google_dir + 'new_users (foreign) top 5000.csv'
pg_views_f = google_dir + 'pg_views (foreign) top 5000.csv'
uniq_pg_views_f = google_dir + 'uniq_pg_views (foreign) top 5000.csv'

new_cols = lambda x: ['year', 'country', 'page', x]

In [48]:
import pandas as pd
pd.options.display.max_colwidth = 100

from matplotlib import pyplot as plt
%matplotlib inline

In [12]:
new_users_df = pd.read_csv(new_users_f, skiprows=5)
new_users_df.columns = new_cols('new_users')
new_users_df.head()

,year,country,page,new_users
0,2017,Bangladesh,Power of the Pen: Identities and Social Issues...,"6,872"
1,2017,India,Power of the Pen: Identities and Social Issues...,"3,537"
2,2017,Nepal,Power of the Pen: Identities and Social Issues...,"1,824"
3,2016,Canada,How Writers Write Fiction 2016: Storied Women ...,"1,493"
4,2017,Pakistan,Power of the Pen: Identities and Social Issues...,"1,317"


In [13]:
pg_views_df = pd.read_csv(pg_views_f, skiprows=5)
pg_views_df.columns = new_cols('pg_views')
pg_views_df.head()

,year,country,page,pg_views
0,2017,Canada,Power of the Pen: Identities and Social Issues...,"47,458"
1,2017,United Kingdom,Power of the Pen: Identities and Social Issues...,"43,822"
2,2017,Australia,Power of the Pen: Identities and Social Issues...,"42,828"
3,2017,India,Power of the Pen: Identities and Social Issues...,"40,521"
4,2016,Canada,How Writers Write Fiction 2016: Storied Women ...,"26,192"


In [14]:
uniq_pg_df = pd.read_csv(uniq_pg_views_f, skiprows=5)
uniq_pg_df.columns = new_cols('uniq_pg_views')
uniq_pg_df.head()

,year,country,page,uniq_pg_views
0,2017,Canada,Power of the Pen: Identities and Social Issues...,"30,893"
1,2017,United Kingdom,Power of the Pen: Identities and Social Issues...,"29,045"
2,2017,Australia,Power of the Pen: Identities and Social Issues...,"28,410"
3,2017,India,Power of the Pen: Identities and Social Issues...,"27,286"
4,2016,Canada,How Writers Write Fiction 2016: Storied Women ...,"19,527"


### Classes



In [40]:
classes_df = pg_views_df.page.value_counts().reset_index()
classes_df.columns = ['page', 'cnt']

In [51]:
# strip the ending
#strip_page_title = lambda x: x.split('|')[0].strip()
strip_page_title = lambda x: x.split('| NovoEd')[0].replace('| The University of Iowa', '').strip()

classes_df['page_clean'] = classes_df.page.apply(strip_page_title)

In [52]:
classes_df[['page', 'page_clean']].iloc[0:500]

,page,page_clean
0,How Writers Write Fiction 2015 | The University of Iowa | NovoEd,How Writers Write Fiction 2015
1,Power of the Pen: Identities and Social Issues in Fiction and Nonfiction | NovoEd,Power of the Pen: Identities and Social Issues in Fiction and Nonfiction
2,How Writers Write Fiction 2016: Storied Women | The University of Iowa | NovoEd,How Writers Write Fiction 2016: Storied Women
3,Power of the Pen: Identities and Social Issues in Poetry and Plays | The University of Iowa | No...,Power of the Pen: Identities and Social Issues in Poetry and Plays
4,Power of the Pen: Identities and Social Issues in Fiction and Nonfiction | The University of Iow...,Power of the Pen: Identities and Social Issues in Fiction and Nonfiction
5,Power of the Pen: Identities and Social Issues in Poetry and Plays | NovoEd,Power of the Pen: Identities and Social Issues in Poetry and Plays
6,Assignments - How Writers Write Fiction 2015 | The University of Iowa | NovoEd,Assignments - How Writers Write Fiction 2015
7,"Whitman’s Civil War: Writing and Imaging Loss, Death, and Disaster | The University of Iowa | No...","Whitman’s Civil War: Writing and Imaging Loss, Death, and Disaster"
8,"Class Session 1 Assignment: Starting with Character, My Submission - How Writers Write Fiction 2...","Class Session 1 Assignment: Starting with Character, My Submission - How Writers Write Fiction 2015"
9,"Whitman's Civil War: Writing and Imaging Loss, Death, and Disaster | The University of Iowa | No...","Whitman's Civil War: Writing and Imaging Loss, Death, and Disaster"
